In [1]:
import h5py
import scipy.io as io
import PIL.Image as Image
import numpy as np
import os
import glob
from matplotlib import pyplot as plt
from scipy.ndimage.filters import gaussian_filter
import scipy
import json
import torchvision.transforms.functional as F
from matplotlib import cm as CM
from image import *
from model import CSRNet
import torch
from torchvision import datasets, transforms

# Define device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Transformation for images
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])

# Root path for the dataset
root = 'D:/Data Science/CSRNet-pytorch/ShanghaiTech_Crowd_Counting_Dataset'

# Paths for train and test data
part_A_train = os.path.join(root, 'part_A_final/train_data', 'images')
part_A_test = os.path.join(root, 'part_A_final/test_data', 'images')
part_B_train = os.path.join(root, 'part_B_final/train_data', 'images')
part_B_test = os.path.join(root, 'part_B_final/test_data', 'images')

# Set of paths to test images
path_sets = [part_A_test]
img_paths = []
for path in path_sets:
    img_paths.extend(glob.glob(os.path.join(path, '*.jpg')))

# Load model and move to the device
model = CSRNet().to(device)

# Load pretrained model weights
checkpoint = torch.load('model_best.pth.tar', map_location=device)
model.load_state_dict(checkpoint['state_dict'])

# Calculate Mean Absolute Error (MAE)
mae = 0
for i in range(len(img_paths)):
    # Load and preprocess image
    img = 255.0 * F.to_tensor(Image.open(img_paths[i]).convert('RGB'))
    img[0, :, :] -= 92.8207477031
    img[1, :, :] -= 95.2757037428
    img[2, :, :] -= 104.877445883
    img = img.to(device)

    # Load ground truth density map
    gt_file = h5py.File(img_paths[i].replace('.jpg', '.h5').replace('images', 'ground_truth'), 'r')
    groundtruth = np.asarray(gt_file['density'])

    # Get model output
    output = model(img.unsqueeze(0))
    mae += abs(output.detach().cpu().sum().item() - np.sum(groundtruth))

    # Print progress
    print(i, mae)

# Print final MAE
print("Mean Absolute Error (MAE):", mae / len(img_paths))


C:\Users\Bathula Shirisha\AppData\Local\Temp\ipykernel_2264\3526844027.py:8: DeprecationWarning: Please use `gaussian_filter` from the `scipy.ndimage` namespace, the `scipy.ndimage.filters` namespace is deprecated.
  from scipy.ndimage.filters import gaussian_filter
C:\Users\Bathula Shirisha\AppData\Local\Temp\ipykernel_2264\3526844027.py:47: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_

FileNotFoundError: [Errno 2] No such file or directory: 'model_best.pth.tar'